In [1]:
import nltk
import random
from nltk.corpus import movie_reviews
from nltk.classify.scikitlearn import SklearnClassifier
from nltk import word_tokenize
import pickle

from sklearn.naive_bayes import MultinomialNB, GaussianNB, BernoulliNB

from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import SVC, LinearSVC, NuSVC

from nltk.classify import ClassifierI
from statistics import mode

In [2]:
class VoteClassifier(ClassifierI):
    def __init__(self, *classifiers):
        self._classifiers = classifiers
    
    def classify(self, features):
        votes = []
        for c in self._classifiers:
            v = c.classify(features)
            votes.append(v)
        return mode(votes)
    
    def confidence(self, features):
        
        votes = []
        for c in self._classifiers:
            v = c.classify(features)
            votes.append(v)
        
        choice_votes = votes.count(mode(votes))
        conf = choice_votes / len(votes)
        return conf
        

In [ ]:
import codecs
short_pos = codecs.open("short_reviews/positive.txt","r",encoding="latin2").read()
short_neg = codecs.open("short_reviews/negative.txt","r", encoding="latin2").read()

short_pos = open("pos_tweets_415.txt","r", encoding="ISO-8859-1").read()
short_neg = open("neg_tweets_415.txt","r", encoding="ISO-8859-1").read()

In [ ]:
short_pos = open("pos_tweets_415.txt","r", encoding="ISO-8859-1").read()
short_neg = open("neg_tweets_415.txt","r", encoding="ISO-8859-1").read()

In [4]:
pos_lines = [line.rstrip('\n') for line in open('Tagged tweets/pos_tagged.txt', 'r', encoding='ISO-8859-1')]
neg_lines = [line.rstrip('\n') for line in open('Tagged tweets/neg_tagged.txt', 'r', encoding='ISO-8859-1')]

documents = []

for r in pos.split("\n"):
    documents.append((r,"pos"))

for r in neg.split("\n"):
    documents.append((r,"neg"))

In [5]:
all_words = []
documents = []

In [6]:
# to select only adjects = J, adverbs = R and verb = V we delted all the other words in the text.

allowed_word_types = ["J"]

for p in pos_lines:
    documents.append((p,"pos"))
    words = word_tokenize(p)
    pos = nltk.pos_tag(words)
    for w in pos:
        if w[1][0] in allowed_word_types:
            all_words.append(w[0].lower())

for p in neg_lines:
    documents.append((p,"neg"))
    words = word_tokenize(p)
    pos = nltk.pos_tag(words)
    for w in pos:
        if w[1][0] in allowed_word_types:
            all_words.append(w[0].lower())

In [31]:
save_documents = open("pickled_algorithms/own_labeled/documents.pickle","wb")
pickle.dump(documents, save_documents)
save_documents.close()

In [7]:
all_words = nltk.FreqDist(all_words)
print(all_words.most_common(100))
print(all_words["bitcoin"])

[('bitcoin', 117), ('ly', 95), ('ã¢', 50), ('new', 47), ('crypto', 46), ('top', 32), ('btc', 28), ('com', 22), ('last', 19), ('usd', 17), ('bulish', 17), ('gl', 17), ('next', 17), ('algory', 17), ('financial', 16), ('coin', 16), ('wil', 15), ('u', 15), ('stelar', 14), ('low', 14), ('major', 14), ('daily', 13), ('first', 13), ('digital', 13), ('short', 12), ('riple', 12), ('uk', 12), ('ap', 12), ('dolar', 11), ('great', 11), ('eth', 10), ('etf', 10), ('update', 10), ('right', 10), ('technical', 10), ('best', 9), ('fre', 9), ('strong', 9), ('global', 9), ('much', 9), ('least', 9), ('sel', 8), ('al', 8), ('january', 8), ('ready', 8), ('future', 8), ('stret', 8), ('net', 8), ('god', 8), ('moderate', 8), ('ltc', 7), ('smart', 7), ('real', 7), ('long', 7), ('regulatory', 7), ('n', 7), ('ful', 7), ('sv', 7), ('posible', 7), ('pic', 7), ('average', 7), ('se', 7), ('invest', 6), ('indian', 6), ('togle', 6), ('bad', 6), ('due', 6), ('ico', 6), ('important', 6), ('win', 6), ('dlvr', 6), ('total',

In [8]:
#commonly used words to train against
word_features = list(all_words.keys())[:5000]

In [32]:
save_word_features = open("pickled_algorithms/own_labeled/word_features.pickle","wb")
pickle.dump(word_features, save_word_features)
save_word_features.close()

In [9]:
def find_features(document):
    words = word_tokenize(document)
    features = {}
    for w in word_features:
        features[w] = (w in words)
    
    return features

In [ ]:
#print((find_features(movie_reviews.words("neg/cv000_29416.txt"))))

In [10]:
featuresets = [(find_features(rev), category) for (rev, category) in documents]

In [ ]:
featuresets[1]

save_featuresets = open("pickled_algorithms/featuresets.pickle","wb")
pickle.dump(featuresets, save_featuresets)
save_featuresets.close()

In [11]:
random.shuffle(featuresets)

In [12]:
training_set = featuresets[:600]
testing_set = featuresets[600:]

In [ ]:
training_set

In [13]:
# likelyhood = prior occurences x likihood / evidence

classifier = nltk.NaiveBayesClassifier.train(training_set)

#classifier_f = open("naivebayes.pickle", "rb")
#classifier = pickle.load(classifier_f)
#classifier_f.close()

print("Original Naive Bayes Algo accuracy:", (nltk.classify.accuracy(classifier, testing_set))*100)
classifier.show_most_informative_features(30)

Original Naive Bayes Algo accuracy: 61.73913043478261
Most Informative Features
                    blog = True              pos : neg    =      6.5 : 1.0
                    bear = True              neg : pos    =      6.2 : 1.0
                   right = True              neg : pos    =      5.5 : 1.0
                    join = True              pos : neg    =      5.1 : 1.0
                    much = True              neg : pos    =      4.9 : 1.0
                   first = True              pos : neg    =      4.5 : 1.0
                   alert = True              pos : neg    =      4.5 : 1.0
                   short = True              neg : pos    =      4.2 : 1.0
                     low = True              neg : pos    =      4.2 : 1.0
                   forex = True              pos : neg    =      3.8 : 1.0
                    info = True              pos : neg    =      3.8 : 1.0
                   worst = True              neg : pos    =      3.6 : 1.0
              utm_me

In [23]:
save_classifier = open("pickled_algorithms/own_labeled/originalnaivebayes.pickle","wb")
pickle.dump(classifier, save_classifier)
save_classifier.close()

In [14]:
# Mulitnominal classifier
MNB_classifier = SklearnClassifier(MultinomialNB())
MNB_classifier.train(training_set)
print("MNB_classifier accuracy:", (nltk.classify.accuracy(MNB_classifier, testing_set))*100)
#MNB_classifier.show_most_informative_features(15)

MNB_classifier accuracy: 61.30434782608696


In [24]:
save_MNB_classifier = open("pickled_algorithms/own_labeled/MNB.pickle","wb")
pickle.dump(MNB_classifier, save_MNB_classifier)
save_MNB_classifier.close()

In [15]:
BernoulliNB_classifier = SklearnClassifier(BernoulliNB())
BernoulliNB_classifier.train(training_set)
print("BernoulliNB_classifier accuracy:", (nltk.classify.accuracy(BernoulliNB_classifier, testing_set))*100)

BernoulliNB_classifier accuracy: 60.0


In [25]:
save_BernoulliNB_classifier = open("pickled_algorithms/own_labeled/BernoulliNB.pickle","wb")
pickle.dump(BernoulliNB_classifier, save_BernoulliNB_classifier)
save_BernoulliNB_classifier.close()

In [16]:
LogisticRegression_classifier = SklearnClassifier(LogisticRegression())
LogisticRegression_classifier.train(training_set)
print("LogisticRegression_classifier accuracy:", (nltk.classify.accuracy(LogisticRegression_classifier, testing_set))*100)

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression_classifier accuracy: 60.43478260869565


In [26]:
save_LogisticRegression_classifier = open("pickled_algorithms/own_labeled/LogisticRegression.pickle","wb")
pickle.dump(LogisticRegression_classifier, save_LogisticRegression_classifier)
save_LogisticRegression_classifier.close()

In [17]:
SGDClassifier_classifier = SklearnClassifier(SGDClassifier())
SGDClassifier_classifier.train(training_set)
print("SGDClassifier_classifier accuracy:", (nltk.classify.accuracy(SGDClassifier_classifier, testing_set))*100)

SGDClassifier_classifier accuracy: 60.0


/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/stochastic_gradient.py:166: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  FutureWarning)


In [27]:
save_SGDClassifier_classifier = open("pickled_algorithms/own_labeled/SGDClassifier.pickle","wb")
pickle.dump(SGDClassifier_classifier, save_SGDClassifier_classifier)
save_SGDClassifier_classifier.close()

In [18]:
SVC_classifier = SklearnClassifier(SVC())
SVC_classifier.train(training_set)
print("SVC_classifier accuracy:", (nltk.classify.accuracy(SVC_classifier, testing_set))*100)

/opt/conda/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


SVC_classifier accuracy: 48.26086956521739


In [28]:
save_SVC_classifier = open("pickled_algorithms/own_labeled/SVC.pickle","wb")
pickle.dump(SVC_classifier, save_SVC_classifier)
save_SVC_classifier.close()

In [19]:
LinearSVC_classifier = SklearnClassifier(LinearSVC())
LinearSVC_classifier.train(training_set)
print("LinearSVC_classifier accuracy:", (nltk.classify.accuracy(LinearSVC_classifier, testing_set))*100)

LinearSVC_classifier accuracy: 61.30434782608696


In [29]:
save_LinearSVC_classifier = open("pickled_algorithms/own_labeled/LinearSVC.pickle","wb")
pickle.dump(LinearSVC_classifier, save_LinearSVC_classifier)
save_LinearSVC_classifier.close()

In [20]:
NuSVC_classifier = SklearnClassifier(NuSVC())
NuSVC_classifier.train(training_set)
print("NuSVC_classifier accuracy:", (nltk.classify.accuracy(NuSVC_classifier, testing_set))*100)

NuSVC_classifier accuracy: 58.69565217391305


In [30]:
save_NuSVC_classifier = open("pickled_algorithms/own_labeled/NuSVC.pickle","wb")
pickle.dump(NuSVC_classifier, save_NuSVC_classifier)
save_NuSVC_classifier.close()

## Vote classifier

In [21]:
voted_classifier = VoteClassifier(classifier,
                                  MNB_classifier,
                                  BernoulliNB_classifier,
                                  LinearSVC_classifier,
                                  NuSVC_classifier,
                                  SGDClassifier_classifier,
                                  LogisticRegression_classifier)

In [22]:
print("voted_classifier accuracy:", (nltk.classify.accuracy(voted_classifier,testing_set))*100)

voted_classifier accuracy: 63.47826086956522


In [ ]:
for name, value in testing_set[5][0].items():
    if value == True:
        print(name)

In [ ]:
print("Classification:", voted_classifier.classify(testing_set[0][0]), "Confidence %:",voted_classifier.confidence(testing_set[0][0])*100)

In [ ]:
print("Classification:", voted_classifier.classify(testing_set[1][0]), "Confidence %:",voted_classifier.confidence(testing_set[1][0])*100)

In [ ]:
print("Classification:", voted_classifier.classify(testing_set[2][0]), "Confidence %:",voted_classifier.confidence(testing_set[2][0])*100)

In [ ]:
print("Classification:", voted_classifier.classify(testing_set[3][0]), "Confidence %:",voted_classifier.confidence(testing_set[3][0])*100)

In [ ]:
print("Classification:", voted_classifier.classify(testing_set[4][0]), "Confidence %:",voted_classifier.confidence(testing_set[4][0])*100)

In [ ]:
print("Classification:", voted_classifier.classify(testing_set[5][0]), "Confidence %:",voted_classifier.confidence(testing_set[5][0])*100)

In [ ]:
def sentiment(text):
    feats = find_features(text)
    
    return voted_classifier.classify(feats)


## Pickling, how to

save_classifier = open("naivebayes.pickle", "wb")
pickle.dump(classifier, save_classifier)
save_classifier.close()